# Importaciones

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import KFold
from sklearn.metrics import precision_score, recall_score, f1_score
from nltk.stem import SnowballStemmer
import unicodedata
import re

# Configuraciones

In [2]:
class Config:
    NEURONAS_OCULTAS = [64, 128, 256, 1024]
    INICIALIZACIONES = ["normal", "xavier"]
    PESADO_TERMINOS = ["tf", "tf-idf"]
    REPRESENTACIONES = ["unigramas", "bigramas", "unigramas_bigramas"]
    PREPROCESAMIENTOS = [
        "normalizar_texto", 
        "normalizar_texto_stopwords", 
        "normalizar_texto_stopwords_stemming"
    ]
    LEARNING_RATES = [0.01, 0.1, 0.5]
    BATCH_SIZES = [16, 32, 64]
    EPOCHS = [100, 300, 500]
    
    RUTAS_DATASETS = {
        "spanish": {
            "train": "./Recursos/hateval_es_train.json",
            "test": "./Recursos/hateval_es_test.json",
            "all": "./Recursos/hateval_es_all.json"
        },
        "english": {
            "train": "./Recursos/hateval_en_train.json",
            "test": "./Recursos/hateval_en_test.json",
            "all": "./Recursos/hateval_en_all.json"
        }
    }

# Clase MLP

In [3]:
# Función de activación sigmoide
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

# Derivada de la sigmoide
def sigmoid_derivative(x):
    return x * (1 - x)

# Establece la semilla para la generación de números aleatorios
def seed(random_state=33):
    np.random.seed(random_state)

# Inicialización de javier
def xavier_initialization(input_size, output_size):
    # ¿En el parametro size es output, input?
    return np.random.normal(scale=np.sqrt(2 / (input_size + output_size)), size=(output_size, input_size))

# Inicialización normal
def normal_initialization(input_size, output_size):
    return np.random.randn(output_size, input_size) * 0.1

# Preprocesado de datos
def preprocesar(ruta):
    datos = pd.read_csv(ruta, header=0)
    datos_crudos = datos.to_numpy()

    x = datos_crudos[:, :-1]
    y = datos_crudos[:, -1:]

    return x, y

# Normalizar los datos
def normalizar_datos(X):
    scaler = StandardScaler()
    return scaler.fit_transform(X)

# Crear mini lotes
def create_minibatches(X, y, batch_size):
    """
    Genera los lotes de datos (batchs) de acuerdo al parámetro batch_size de forma aleatoria para el procesamiento. 
    """
    n_samples = X.shape[0]
    indices = np.random.permutation(n_samples)  # Mezcla los índices aleatoriamente
    X_shuffled, y_shuffled = X[indices], y[indices]  # Reordena X e y según los índices aleatorios
    
    # Divide los datos en minibatches
    for X_batch, y_batch in zip(np.array_split(X_shuffled, np.ceil(n_samples / batch_size)), 
                                np.array_split(y_shuffled, np.ceil(n_samples / batch_size))):
        yield X_batch, y_batch

# Probar modelo
def evaluar_modelo_prueba(modelo, ruta_prueba, normalizar):
    x_test_crudo, Y_test_crudo = preprocesar(ruta_prueba)

    if normalizar:
        x_test = normalizar_datos(x_test_crudo)
    else:
        x_test = x_test_crudo
    
    metricas = modelo.evaluar(x_test, Y_test_crudo)

    return metricas


class MLP_TODO:
    def __init__(self, num_entradas, num_neuronas_ocultas, num_salidas, epochs, batch_size=128, learning_rate=0.2, random_state=42, initialization="xavier"):

        # Construcción
        seed(random_state)
        self.learning_rate = learning_rate
        self.epochs = epochs
        self.batch_size = batch_size
        
        self.error_mse = []
        self.accuracy_epoca = []
        
        # definir las capas
        if initialization == 'xavier':
            init_fun = xavier_initialization
        else : 
            init_fun = normal_initialization

        self.W1 = init_fun(num_entradas, num_neuronas_ocultas)
        self.b1 = np.zeros((1, num_neuronas_ocultas))
        self.W2 = init_fun(num_neuronas_ocultas, num_salidas)
        self.b2 = np.zeros((1, num_salidas))

    def forward(self, X):
        #----------------------------------------------
        # 1. Propagación hacia adelante (Forward pass)
        #----------------------------------------------
        self.X = X
        self.z_c1 = X @ self.W1.T + self.b1
        self.a_c1 = sigmoid(self.z_c1)
        self.z_c2 = self.a_c1 @ self.W2.T + self.b2
        y_pred = sigmoid(self.z_c2)  # Y^
        return y_pred

    def loss_function_MSE(self, y_pred, y):
        #----------------------------------------------
        # 2. Cálculo del error con MSE
        #----------------------------------------------
        self.y_pred = y_pred
        self.y = y
        error = 0.5 * np.mean((y_pred - y) ** 2)
        return error
    
    def backward(self):
        #----------------------------------------------
        # 3. Propagación hacia atrás (Backward pass)
        #----------------------------------------------
        
        #----------------------------------------------
        # Gradiente de la salida
        #----------------------------------------------
        dE_dy_pred = (self.y_pred - self.y) / self.y.shape[0] # Derivada del error respecto a la predicción con  N ejemplos
        d_y_pred_d_zc2 = sigmoid_derivative(self.y_pred)
        delta_c2 = dE_dy_pred * d_y_pred_d_zc2

        #----------------------------------------------
        # Gradiente en la capa oculta
        #----------------------------------------------
        # calcular la derivada de las suma ponderada respecto a las activaciones de la capa 1
        delta_c1 = (delta_c2 @ self.W2) * sigmoid_derivative(self.a_c1)

        #calcula el gradiente de pesos y bias
        self.dE_dW2 = delta_c2.T @ self.a_c1
        self.dE_db2 = np.sum(delta_c2, axis=0, keepdims=True)
        self.dE_dW1 = delta_c1.T @ self.X
        self.dE_db1 = np.sum(delta_c1, axis=0, keepdims=True)

    def update(self):  # Ejecución de la actualización de paramámetros
        #----------------------------------------------
        # Actualización de pesos de la capa de salida
        #---------------------------------------------- 
        
        self.W2 = self.W2 - self.learning_rate * self.dE_dW2 # Ojito con la T
        self.b2 = self.b2 - self.learning_rate * self.dE_db2

        #----------------------------------------------
        # Actuailzación de pesos de la capa oculta
        #----------------------------------------------
        #calcula el gradiente de la función de error respecto a los pesos de la capa 1
        self.W1 = self.W1 - self.learning_rate * self.dE_dW1
        self.b1 = self.b1 - self.learning_rate * self.dE_db1

    def predict(self, X):  # Predecir la categoría para datos nuevos
        y_pred = self.forward(X)
        # Obtener la clase para el clasificador binario
        y_pred = np.where(y_pred >= 0.5, 1, 0)
        return y_pred

    def train(self, X, Y):
        for epoch in range(self.epochs):

            num_batch = 0
            epoch_error  = 0

            # Procesamiento por lotes
            for X_batch, y_batch in create_minibatches(X, Y, self.batch_size):
                y_pred = self.forward(X_batch)
                error = self.loss_function_MSE(y_pred, y_batch)
                
                # if np.all(y_pred == Y) : aciertos += 1
                # self.accuracy_epoca.append(aciertos/epoch)

                epoch_error += error
                self.backward() # cálculo de los gradientes
                self.update() # actualización de los pesos y bias
                num_batch += 1
                # Imprimir el error cada N épocas
            
            # Almacena el error promedio por época
            self.error_mse.append(epoch_error/num_batch)

            # Obtener predicciones binarias para todo el conjunto de entrenamiento
            y_pred_total = self.predict(X)

            # Calcular la exactitud
            exactitud = self.calcular_accuracy(y_pred_total, Y) 
            
            # Almacenar la exactitud de la época
            self.accuracy_epoca.append(exactitud)

            #if epoch % 100 == 0: print(f"Época {epoch:05d} | MSE: {epoch_error/num_batch:.6f} | Exactitud: {exactitud:.4f}")

    def graficar(self, graficar_exactitud=True, guardar=True, nombre="grafica"):
        """ 
        Para MSE siempre se muestra 
        """
        # Preparar datos
        mse = np.arange(len(self.error_mse))

        # Crear tabla
        plt.figure(figsize=(10,6))

        #Graficar MSE
        plt.plot(mse, self.error_mse, label="MSE", color="green", linewidth=1)


        """ 
        Para la exactitud 
        """
        if graficar_exactitud and len(self.accuracy_epoca) > 0:
            accuracy = np.arange(len(self.accuracy_epoca))
            plt.plot(accuracy, self.accuracy_epoca, label="Exactitud", color="green", linewidth=1)
            plt.ylabel("MSE / Exactitud")
            titulo = "Evolución del Error (MSE) y Exactitud durante el entrenamiento"
        else:
            plt.ylabel("Error Cuadrático Medio (MSE)")
            titulo = "Evolución del Error (MSE) durante el entrenamiento"

        plt.title(titulo)
        plt.xlabel("Época")
        plt.legend()
        plt.grid(True, alpha=0.3)

        if guardar:
            plt.savefig(f'./graficas/{nombre}.svg')
        plt.show()

    def calcular_accuracy(self, y_pred, y_verdadera):
        return np.mean(y_verdadera.flatten() == y_pred.flatten())

    def analizar(self, X, y):
        # Gráficar
        self.graficar(guardar=True, graficar_exactitud=False)

        # Valores reales y predicción
        y_pred = self.predict(X)
        print(f"valores reales: {y.flatten()}")
        print(f"Predicciones  : {y_pred.flatten()}")

        # Calcular exactitud
        exactitud = self.calcular_accuracy(y_pred, y)
        print(f"Exactitud: {exactitud}")

    def evaluar(self, x_test, y_test):
        y_gorrito = self.predict(x_test)

        accuracy = self.calcular_accuracy(y_gorrito, y_test)

        probabilidad = self.forward(x_test)
        mse = self.loss_function_MSE(probabilidad, y_test)

        metricas = {
            "Exactitud": accuracy,
            "mse": mse
        }

        return metricas

# Procesamiento de texto

In [4]:
class PreprocesadorTexto:
    def __init__(self, eliminar_stopwords=True, aplicar_stemming=True, idioma="spanish"):
        self.eliminar_stopwords = eliminar_stopwords
        self.aplicar_stemming = aplicar_stemming
        self.idioma = idioma
        self.stemmer = SnowballStemmer(idioma) if aplicar_stemming else None
        
    def normalizar_texto(self, texto, eliminar_numeros=True, eliminar_puntuacion=True):
        """Normaliza el texto eliminando acentos, números y puntuación"""
        if pd.isna(texto):
            return ""
            
        # Normalizar Unicode
        texto = unicodedata.normalize('NFKD', str(texto))
        
        # Eliminar acentos
        texto = ''.join(c for c in texto if not unicodedata.combining(c))
        
        # Eliminar números
        if eliminar_numeros:
            texto = re.sub(r'\d+', '', texto)
        
        # Eliminar puntuación
        if eliminar_puntuacion:
            texto = re.sub(r'[^\w\s]', '', texto)
        
        # Convertir a minúsculas y eliminar espacios extras
        texto = texto.lower().strip()
        texto = re.sub(r'\s+', ' ', texto)
        
        return texto
    
    def aplicar_preprocesamiento(self, df, columna_texto="text"):
        """Aplica todo el pipeline de preprocesamiento"""
        df_procesado = df.copy()
        
        # Normalización básica
        df_procesado[columna_texto] = df_procesado[columna_texto].apply(self.normalizar_texto)
        
        # Stemming
        if self.aplicar_stemming:
            df_procesado[columna_texto] = df_procesado[columna_texto].apply(
                lambda x: ' '.join([self.stemmer.stem(palabra) for palabra in x.split()])
            )
        
        return df_procesado

# Generación de representaciones vectoriales

In [5]:
# Celda 3: Generación de representaciones vectoriales
class GeneradorRepresentaciones:
    def __init__(self, tipo_representacion="tf", n_gramas=(1, 1)):
        self.tipo_representacion = tipo_representacion
        self.n_gramas = n_gramas
        self.vectorizador = None
        self.vocabulario = None
        
    def fit_transform(self, textos):
        """Entrena el vectorizador y transforma los textos"""
        if self.tipo_representacion == "tf-idf":
            self.vectorizador = TfidfVectorizer(ngram_range=self.n_gramas)
        else:  # TF por defecto
            self.vectorizador = CountVectorizer(ngram_range=self.n_gramas)
            
        X = self.vectorizador.fit_transform(textos).toarray()
        self.vocabulario = self.vectorizador.get_feature_names_out()
        
        return X
    
    def transform(self, textos):
        """Transforma nuevos textos usando el vectorizador entrenado"""
        if self.vectorizador is None:
            raise ValueError("El vectorizador debe ser entrenado primero con fit_transform")
        
        return self.vectorizador.transform(textos).toarray()

# Experimentación

In [6]:
# Celda 4: Gestión de experimentos
class GestorExperimentos:
    def __init__(self, config):
        self.config = config
        self.resultados = []
    
    def generar_combinaciones(self):
        """Genera todas las combinaciones de parámetros a probar"""
        from itertools import product
        
        combinaciones = list(product(
            self.config.NEURONAS_OCULTAS,
            self.config.INICIALIZACIONES,
            self.config.PESADO_TERMINOS,
            self.config.REPRESENTACIONES,
            self.config.PREPROCESAMIENTOS,
            self.config.LEARNING_RATES,
            self.config.BATCH_SIZES,
            self.config.EPOCHS,
            ["spanish", "english"]  # datasets
        ))
        
        return combinaciones
    
    def ejecutar_experimento(self, combinacion, datos_preprocesados, matrices_vsm):
        """Ejecuta un experimento individual"""
        (neuronas_ocultas, inicializacion, pesado, representacion, 
         preprocesamiento, learning_rate, batch_size, epochs, idioma) = combinacion
        
        print(f"Ejecutando: {idioma}, neuronas={neuronas_ocultas}, lr={learning_rate}, batch={batch_size}")
        
        try:
            # Preparar datos según la combinación
            X_train = matrices_vsm[idioma][f"train_{pesado}"]
            y_train = datos_preprocesados[idioma]["train"]["klass"].values
            X_test = matrices_vsm[idioma][f"test_{pesado}"] 
            y_test = datos_preprocesados[idioma]["test"]["klass"].values
            
            # Crear y entrenar modelo
            modelo = MLP_TODO(
                num_entradas=X_train.shape[1],
                num_neuronas_ocultas=neuronas_ocultas,
                num_salidas=1,
                epochs=epochs,
                batch_size=batch_size,
                learning_rate=learning_rate,
                initialization=inicializacion
            )
            
            modelo.train(X_train, y_train.reshape(-1, 1))
            
            # Evaluar
            metricas = modelo.evaluar(X_test, y_test.reshape(-1, 1))
            
            # Almacenar resultados
            resultado = {
                "neuronas_ocultas": neuronas_ocultas,
                "inicializacion": inicializacion,
                "pesado": pesado,
                "representacion": representacion,
                "preprocesamiento": preprocesamiento,
                "learning_rate": learning_rate,
                "batch_size": batch_size,
                "epochs": epochs,
                "idioma": idioma,
                "exactitud": metricas["Exactitud"],
                "mse": metricas["mse"],
                "precision": metricas["precision"],
                "recall": metricas["recall"],
                "f1_score": metricas["f1_score"],
                "modelo": modelo  # Guardar referencia al modelo para validación cruzada
            }
            
            self.resultados.append(resultado)
            return resultado
            
        except Exception as e:
            print(f"Error en experimento {combinacion}: {e}")
            return None
    
    def validacion_cruzada(self, mejor_resultado, datos_completos, k_folds=5):
        """Ejecuta validación cruzada para las mejores configuraciones"""
        idioma = mejor_resultado["idioma"]
        X = datos_completos[idioma]["all"]
        y = datos_completos[idioma]["all"]["klass"].values
        
        kf = KFold(n_splits=k_folds, shuffle=True, random_state=42)
        metricas_folds = []
        
        for fold, (train_index, val_index) in enumerate(kf.split(X)):
            print(f"Fold {fold + 1}/{k_folds}")
            
            X_train, X_val = X[train_index], X[val_index]
            y_train, y_val = y[train_index], y[val_index]
            
            # Crear nuevo modelo con misma configuración
            modelo = MLP_TODO(
                num_entradas=X_train.shape[1],
                num_neuronas_ocultas=mejor_resultado["neuronas_ocultas"],
                num_salidas=1,
                epochs=mejor_resultado["epochs"],
                batch_size=mejor_resultado["batch_size"],
                learning_rate=mejor_resultado["learning_rate"],
                initialization=mejor_resultado["inicializacion"]
            )
            
            # Entrenar y evaluar
            modelo.train(X_train, y_train.reshape(-1, 1))
            y_pred = modelo.predict(X_val)
            
            precision = precision_score(y_val, y_pred.flatten(), zero_division=0)
            recall = recall_score(y_val, y_pred.flatten(), zero_division=0)
            f1 = f1_score(y_val, y_pred.flatten(), zero_division=0)
            accuracy = np.mean(y_val == y_pred.flatten())
            
            metricas_folds.append({
                "fold": fold + 1,
                "precision": precision, 
                "recall": recall, 
                "f1": f1,
                "accuracy": accuracy
            })
        
        return metricas_folds

# Carga y preprocesamiento

In [7]:
# Celda 5: Carga y preprocesamiento de datos
config = Config()

# Cargar datos
datos = {}
for idioma, archivos in config.RUTAS_DATASETS.items():
    datos[idioma] = {}
    for tipo, ruta in archivos.items():
        datos[idioma][tipo] = pd.read_json(ruta, lines=True)
        print(f"Cargando {idioma}_{tipo}, {len(datos[idioma][tipo])} datos")

# Preprocesamiento básico
preprocesador = PreprocesadorTexto(aplicar_stemming=True)
datos_preprocesados = {}

for idioma in datos:
    datos_preprocesados[idioma] = {}
    for tipo in datos[idioma]:
        datos_preprocesados[idioma][tipo] = preprocesador.aplicar_preprocesamiento(
            datos[idioma][tipo]
        )
        print(f"Preprocesado {idioma}_{tipo}")

print("Muestra de datos preprocesados:")
print(datos_preprocesados["spanish"]["train"].head(3))

Cargando spanish_train, 4500 datos
Cargando spanish_test, 500 datos
Cargando spanish_all, 5000 datos
Cargando english_train, 9000 datos
Cargando english_test, 1000 datos
Cargando english_all, 10000 datos
Preprocesado spanish_train
Preprocesado spanish_test
Preprocesado spanish_all
Preprocesado english_train
Preprocesado english_test
Preprocesado english_all
Muestra de datos preprocesados:
      id  klass                                               text
0  20001      1  easyjet quier duplic el numer de mujer pilot v...
1  20002      1  el gobiern deb cre un control estrict de inmig...
2  20003      0  yo veo a mujer destru por acos laboral y calle...


# Generación de representaciones vectoriales

In [8]:
# Celda 6: Generación de representaciones vectoriales
matrices_vsm = {}

for idioma in datos_preprocesados:
    matrices_vsm[idioma] = {}
    
    textos_train = datos_preprocesados[idioma]["train"]["text"].tolist()
    textos_test = datos_preprocesados[idioma]["test"]["text"].tolist()
    
    # Para cada tipo de pesado (TF y TF-IDF)
    for tipo_pesado in ["tf", "tf-idf"]:
        print(f"Generando {tipo_pesado} para {idioma}")
        
        generador = GeneradorRepresentaciones(tipo_representacion=tipo_pesado)
        X_train = generador.fit_transform(textos_train)
        X_test = generador.transform(textos_test)
        
        matrices_vsm[idioma][f"train_{tipo_pesado}"] = X_train
        matrices_vsm[idioma][f"test_{tipo_pesado}"] = X_test
        
        print(f"  {idioma} {tipo_pesado}: train {X_train.shape}, test {X_test.shape}")

Generando tf para spanish
  spanish tf: train (4500, 12868), test (500, 12868)
Generando tf-idf para spanish
  spanish tf-idf: train (4500, 12868), test (500, 12868)
Generando tf para english
  english tf: train (9000, 24214), test (1000, 24214)
Generando tf-idf para english
  english tf-idf: train (9000, 24214), test (1000, 24214)


# Ejecución de experimentos

In [ ]:
# Celda 7: Ejecución de experimentos (versión reducida para prueba)
gestor = GestorExperimentos(config)

# Generar combinaciones (puedes reducir para pruebas)
combinaciones = gestor.generar_combinaciones()
print(f"Total de combinaciones: {len(combinaciones)}")

# Probar solo algunas combinaciones para demo
combinaciones_prueba = combinaciones[:5]  # Solo 5 para prueba rápida

print("Ejecutando experimentos de prueba...")
for i, combinacion in enumerate(combinaciones_prueba):
    print(f"\n--- Experimento {i+1}/{len(combinaciones_prueba)} ---")
    resultado = gestor.ejecutar_experimento(combinacion, datos_preprocesados, matrices_vsm)
    if resultado:
        print(f"Resultado: F1-score = {resultado['f1_score']:.4f}, Exactitud = {resultado['exactitud']:.4f}")

# Mostrar resultados
if gestor.resultados:
    resultados_df = pd.DataFrame(gestor.resultados)
    print("\nResultados obtenidos:")
    print(resultados_df[['idioma', 'neuronas_ocultas', 'learning_rate', 'exactitud', 'f1_score']])

Total de combinaciones: 7776
Ejecutando experimentos de prueba...

--- Experimento 1/5 ---
Ejecutando: spanish, neuronas=64, lr=0.01, batch=16
Error en experimento (64, 'normal', 'tf', 'unigramas', 'normalizar_texto', 0.01, 16, 100, 'spanish'): 'precision'

--- Experimento 2/5 ---
Ejecutando: english, neuronas=64, lr=0.01, batch=16


# Análisis de resultados y visualización

In [ ]:
# Celda 8: Análisis de resultados y visualización
def analizar_resultados(gestor):
    if not gestor.resultados:
        print("No hay resultados para analizar")
        return
    
    resultados_df = pd.DataFrame(gestor.resultados)
    
    # Mejores configuraciones por F1-score
    mejores_5 = resultados_df.nlargest(5, 'f1_score')
    
    print("=== MEJORES 5 CONFIGURACIONES ===")
    print(mejores_5[['idioma', 'neuronas_ocultas', 'inicializacion', 'pesado', 
                     'learning_rate', 'batch_size', 'f1_score', 'exactitud']])
    
    # Gráficas
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # 1. F1-score por idioma
    resultados_df.groupby('idioma')['f1_score'].mean().plot(kind='bar', ax=axes[0,0], title='F1-score promedio por idioma')
    
    # 2. F1-score por número de neuronas
    resultados_df.groupby('neuronas_ocultas')['f1_score'].mean().plot(kind='bar', ax=axes[0,1], title='F1-score por neuronas ocultas')
    
    # 3. F1-score por inicialización
    resultados_df.groupby('inicializacion')['f1_score'].mean().plot(kind='bar', ax=axes[1,0], title='F1-score por tipo de inicialización')
    
    # 4. F1-score por learning rate
    resultados_df.groupby('learning_rate')['f1_score'].mean().plot(kind='bar', ax=axes[1,1], title='F1-score por learning rate')
    
    plt.tight_layout()
    plt.show()
    
    return mejores_5

# Ejecutar análisis
mejores_configuraciones = analizar_resultados(gestor)

# Validación cruzada para mejores configuraciones

In [ ]:
# Celda 9: Validación cruzada para mejores configuraciones
def ejecutar_validacion_cruzada(gestor, datos_completos, mejores_configuraciones, k_folds=5):
    print("=== VALIDACIÓN CRUZADA PARA MEJORES CONFIGURACIONES ===\n")
    
    resultados_cv = {}
    
    for i, (idx, configuracion) in enumerate(mejores_configuraciones.iterrows()):
        print(f"Configuración {i+1}: {configuracion['idioma']}, neuronas={configuracion['neuronas_ocultas']}, F1={configuracion['f1_score']:.4f}")
        
        metricas_cv = gestor.validacion_cruzada(configuracion, datos_completos, k_folds)
        
        # Resumen de validación cruzada
        df_cv = pd.DataFrame(metricas_cv)
        promedio_cv = df_cv.mean()
        
        print(f"  Validación cruzada (promedio):")
        print(f"    Exactitud: {promedio_cv['accuracy']:.4f}")
        print(f"    Precisión: {promedio_cv['precision']:.4f}")
        print(f"    Recall: {promedio_cv['recall']:.4f}")
        print(f"    F1-score: {promedio_cv['f1']:.4f}")
        print()
        
        resultados_cv[i] = {
            'configuracion': configuracion.to_dict(),
            'validacion_cruzada': metricas_cv,
            'promedio_cv': promedio_cv
        }
    
    return resultados_cv

# Ejecutar validación cruzada si hay resultados
if not mejores_configuraciones.empty:
    # Preparar datos completos para validación cruzada
    datos_completos = {}
    for idioma in datos_preprocesados:
        datos_completos[idioma] = {}
        # Combinar train y test para validación cruzada
        df_all = pd.concat([datos_preprocesados[idioma]["train"], datos_preprocesados[idioma]["test"]])
        datos_completos[idioma]["all"] = df_all.reset_index(drop=True)
    
    resultados_cv = ejecutar_validacion_cruzada(gestor, datos_completos, mejores_configuraciones.head(3))